In [1]:
# Importar bibliotecas
import pandas as pd
import socket
from dns import resolver
from ipwhois import IPWhois
from geopy.geocoders import Nominatim

# Função para obter DNS a partir da URL
def get_dns_from_url(url):
    domain = url.split('//')[-1].split('/')[0]
    try:
        answers = resolver.resolve(domain, 'A')
        ips = [str(rdata) for rdata in answers]
        return ips
    except Exception as e:
        return str(e)

# Função para obter informações do IP
def get_ip_info(ip):
    try:
        obj = IPWhois(ip)
        results = obj.lookup_rdap(asn_methods=["whois", "http"])
        return results
    except Exception as e:
        return str(e)

# Função para obter informações geográficas do IP
def get_geolocation(ip):
    try:
        geolocator = Nominatim(user_agent="geoapiExercises")
        location = geolocator.geocode(ip)
        if location:
            return location.address
        else:
            return "Location not found"
    except Exception as e:
        return str(e)

# Ler o dataset
data_url = 'PhishingDataset_copy.csv'  # Substitua pelo caminho do seu dataset
df = pd.read_csv(data_url, delimiter=';', header=0, encoding='utf-8')

# Inicializar colunas para os resultados
df['DNS'] = None
df['IP'] = None
df['ASN'] = None
df['GeoLocation'] = None

# Iterar sobre as URLs no dataset e obter as informações necessárias
for index, row in df.iterrows():
    url = row['URL']
    
    # Obter DNS e IP
    dns_ips = get_dns_from_url(url)
    if isinstance(dns_ips, list) and dns_ips:
        ip = dns_ips[0]  # Pegar o primeiro IP da lista
        df.at[index, 'DNS'] = ", ".join(dns_ips)
        df.at[index, 'IP'] = ip

        # Obter informações do IP
        ip_info = get_ip_info(ip)
        df.at[index, 'ASN'] = ip_info.get('asn', 'N/A')
        
        # Obter localização geográfica
        geo_info = get_geolocation(ip)
        df.at[index, 'GeoLocation'] = geo_info
    else:
        df.at[index, 'DNS'] = dns_ips  # Erro ou mensagem de falha

# Mostrar as primeiras linhas do DataFrame atualizado
print(df[['URL', 'DNS', 'IP', 'ASN', 'GeoLocation']].head())

# Imprimir todas as informações
for index, row in df.iterrows():
    print(f"URL: {row['URL']}")
    print(f"DNS: {row['DNS']}")
    print(f"IP: {row['IP']}")
    print(f"ASN: {row['ASN']}")
    print(f"GeoLocation: {row['GeoLocation']}")
    print("-" * 40)


ModuleNotFoundError: No module named 'dns'